In [5]:
# importar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [7]:
# cargar archivo

df = pd.read_csv("data.csv")

df.shape

(10000, 26)

In [14]:
# Separar datos 80 - 20

X = df.drop('Attrition', axis=1)
y = df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2000